다음 뉴스 URL, 제목 

In [1]:
import requests
import lxml.html
import pandas as pd

In [14]:
pd.date_range('2016-01-01', '2017-01-01')

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2016-12-23', '2016-12-24', '2016-12-25', '2016-12-26',
               '2016-12-27', '2016-12-28', '2016-12-29', '2016-12-30',
               '2016-12-31', '2017-01-01'],
              dtype='datetime64[ns]', length=367, freq='D')

In [12]:
response = requests.get('https://news.daum.net/breakingnews/digital?regDate=20200505')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('a'):
    url = a.get('href')
    print(url, a.text)

#kakaoBody 본문 바로가기
#kakaoGnb 메뉴 바로가기
https://news.daum.net/ None
https://entertain.daum.net None
https://sports.media.daum.net/sports None
/ None
/society None
/politics None
/economic None
/foreign None
/culture None
/digital None
/ranking/popular None
/series/ None
/photo None
/tv/ None
/1boon None
https://gallery.v.daum.net/p/home None
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=서울특별시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=수원시 권선구날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=인천광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=대구광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=대전광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=광주광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=부산광역시날씨 
										
http://search.daum.net/search?w=tot&DA=23W&rtmaxcoll=Z8T&q=울산광역시날씨 
										
http://search.daum.net/se